In [ ]:
import os
import sys
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

In [ ]:
import vso.util
import vso.data

OBJ_NAME='SZ Lyn'
SESSION_TAG='20250126'
# OBJ_NAME='RZ Lyr'
# SESSION_TAG='20240827'
IMAGE_ROOT = '/srv/public/img'
WORK_ROOT = '/srv/public'

session = vso.util.Session(tag=SESSION_TAG, name=OBJ_NAME)
layout = vso.util.WorkLayout(WORK_ROOT)
session_layout = layout.get_session(session)
settings = vso.util.Settings(session_layout.settings_file_path)

In [ ]:
from vso import phot
from astropy.table import join

BANDS = ('B', 'V')

provider = phot.BatchDataProvider(session_layout)

sequence = provider.sequence_band_pair(BANDS)
# measured = provider.batch_BANDSS_pair(BANDS)
# data = join(join(measured, sequence, 'auid'), provider.batches_['batch_id', 'airmass'], 'batch_id')
# sequence.meta
COMP_STAR = '000-BJR-415'
CHECK_STAR = '000-BJR-416'
TARGET_STAR = '000-BBP-158'
comp = provider.batch_comp_star(BANDS, COMP_STAR)
targ = provider.batch_target_star(BANDS,  provider.target_auid)
xfm_input = join(join(comp, targ, 'batch_id'), provider.batches_['batch_id', 'airmass', 'time'], 'batch_id')
xfm_input

In [ ]:
provider.sequence_.meta

In [ ]:
import numpy as np
from vso.util import MagErr, MagErrDtype, ValErr
from vso.phot import ClassicDiffTransform

xfm = ClassicDiffTransform(band=('B', 'V'),
                           Ta=ValErr(val=np.float32(-0.14637248), err=np.float32(0.022204187)),
                           ka=ValErr(val=np.float32(-0.25006065), err=np.float32(0.01773947)),
                           Tb=ValErr(val=np.float32(-0.056993328), err=np.float32(0.015422444)),
                           kb=ValErr(val=np.float32(-0.07465822), err=np.float32(0.012321368)))
# xfm = ClassicDiffTransform(band=('B', 'V'),
#                            Ta=ValErr(val=np.float32(-0.027707625), err=np.float32(0.047621105)),
#                            ka=ValErr(val=np.float32(-0.22594203), err=np.float32(0.044250127)),
#                            Tb=ValErr(val=np.float32(-0.074448444), err=np.float32(0.042286843)),
#                            kb=ValErr(val=np.float32(-0.085416794), err=np.float32(0.03929346)))

In [ ]:
import astropy.units as u
from astropy.table import QTable, Column


def apply_classic_diff_transform(data, bands):
    def apply_xfm(xfm, A_c, B_c, a_c, b_c, a_t, b_t, am):
        Ac, Ac_err = A_c
        Bc, Bc_err = B_c
        ac, ac_err = a_c
        bc, bc_err = b_c
        at, at_err = a_t
        bt, bt_err = b_t

        _, (Ta, Ta_err), (k2a, k2a_err), (Tb, Tb_err), (k2b, k2b_err) = xfm

        ct, ct_err = at-bt, np.sqrt(at_err**2 + bt_err**2)
        cc, cc_err = ac-bc, np.sqrt(ac_err**2 + bc_err**2)
        dc, dc_err = ct-cc, np.sqrt(ct_err**2 + cc_err**2)
        da, da_err = at-ac, np.sqrt(at_err**2 + ac_err**2)
        db, db_err = at-ac, np.sqrt(bt_err**2 + bc_err**2)

        K = 1/(1 - ((Ta-Tb) - am*(k2a-k2b)))
        K_err = np.sqrt(Ta_err**2 + Tb_err**2 + am*(k2a_err**2 + k2b_err))

        dC = K * dc
        dC_err = K * dc*np.sqrt((K_err/K)**2 + (dc_err/dc)**2)

        At = Ac + da - k2a * am * dC + Ta * dC
        At_err = np.sqrt(Ac_err**2
                         + da_err ** 2
                         + (k2a * am*dC * np.sqrt((k2a_err/k2a)**2
                                                  + (dC_err/dC)**2))**2
                         + (Ta*dC * np.sqrt((Ta_err/Ta)**2
                                            + (dC_err/dC)**2))**2
                         )
        Bt = Bc + db - k2b * am * dC + Tb * dC
        Bt_err = np.sqrt(Bc_err**2
                         + db_err**2
                         + (k2b * am*dC * np.sqrt((k2b_err/k2b)**2
                                                  + (dC_err/dC)**2))**2
                         + (Tb*dC * np.sqrt((Tb_err/Tb)**2
                                            + (dC_err/dC)**2))**2
                         )
        return MagErr(At, At_err), MagErr(Bt, Bt_err)

    #bands = BANDS
    transformed = [(row['time'], apply_xfm(xfm, row[bands[0]].value, row[bands[1]].value,
                                row[provider.instr(bands[0])].value, row[provider.instr(bands[1])].value,
                                row[provider.targ(bands[0])].value, row[provider.targ(bands[1])].value,
                                row['airmass']
                                )) for row in data]

    time, M = zip(*transformed)
    At, Bt = zip(*M)
    result = QTable({
        'time': list(time),
        BANDS[0]: Column(list(At), unit = u.mag, dtype=MagErrDtype),
        BANDS[1]: Column(list(Bt), unit = u.mag, dtype=MagErrDtype)
    })
    return result

result = apply_classic_diff_transform(xfm_input, BANDS)

In [ ]:
check = provider.batch_target_star(BANDS, CHECK_STAR)
xfm_input_check = join(join(comp, check, 'batch_id'), provider.batches_['batch_id', 'airmass', 'time'], 'batch_id')
result_check = apply_classic_diff_transform(xfm_input_check, BANDS)

In [ ]:
import matplotlib.pyplot as plt

BAND_COLOR = dict(B='blue',
                  V='green',
                  Rc='red',
                  Ic='magenta')

star = provider.sequence_.meta['star']

check_data = provider.check_band_pair(BANDS, CHECK_STAR)

def plot_band(ax, data, band):
    ax.errorbar(data['time'].jd, data[f'{band}']['mag'],
                yerr=data[f'{band}']['err']/2, fmt='.',
                label=band, color=BAND_COLOR[band])

fig = plt.figure(figsize=(10.24, 10.24))
gs = fig.add_gridspec(3, 1)

ax = fig.add_subplot(gs[0, 0])
plot_band(ax, result, BANDS[0])
plot_band(ax, result, BANDS[1])
ax.invert_yaxis()
# fmter = ScalarFormatter()
# fmter.set_powerlimits((-4, 8))
# ax.xaxis.set_major_formatter(fmter)
ax.legend()
ax.set_xlabel('JD')
ax.set_ylabel('Magnitude')
ax.set_title(f'{star} Light Curves ${{{BANDS[0]}}}$ and ${{{BANDS[1]}}}$')

ax = fig.add_subplot(gs[1, 0])
# ax.xaxis.set_major_formatter(fmter)
ax.plot(result['time'].jd, (result[BANDS[0]]['err']),
            '.', label=f'Target {BANDS[0]}', color=BAND_COLOR[BANDS[0]])
ax.plot(result['time'].jd, (result[BANDS[1]]['err']),
            '.', label=f'Target {BANDS[1]}', color=BAND_COLOR[BANDS[1]])
ax.set_xlabel('JD')
ax.set_ylabel('Target uncertainty')
ax.set_title(f'{star} Photometry errors for ${{{BANDS[0]}}}$ and ${{{BANDS[1]}}}$')

ax = fig.add_subplot(gs[2, 0])
ax.plot(result['time'].jd, ((result_check[BANDS[0]]['mag'] - check_data[BANDS[0]]['mag'])),
            '+', label=f'Target {BANDS[0]}', color=BAND_COLOR[BANDS[0]])
ax.plot(result['time'].jd, ((result_check[BANDS[1]]['mag'] - check_data[BANDS[1]]['mag'])),
            '+', label=f'Target {BANDS[1]}', color=BAND_COLOR[BANDS[1]])
ax.set_ylabel('Check star error')

plt.tight_layout()
plt.show()
